# Compare call clusters
Here, we use silhouette score and kruskal-wallis H test to evaluate whether any of the labelled predictors can describe distribution of data in the model.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import pandas as pd

In [2]:
import avgn

In [3]:
from avgn.utils.paths import DATA_DIR, most_recent_subdirectory, ensure_dir, FIGURE_DIR
from avgn.visualization.spectrogram import draw_spec_set
from avgn.utils.general import save_fig

In [4]:
from scipy.stats import kruskal
from sklearn.metrics import silhouette_score, silhouette_samples

In [5]:
DATASET_ID = "git_repos_call"

In [6]:
DT_ID = '2022-03-12_17-46-00'

## Call Dataframe

In [7]:
call_df = pd.read_pickle(DATA_DIR / DATASET_ID / DT_ID /  'call_umap.pickle')
call_df[:3]

,start_time,end_time,labels,indv,indvi,filename,group,location,sex,wav_loc,...,rate,comb_labels,umap,spectrogram,call_lab_simp,combi_lab_simp,call_unique_num,call_pos_combi,combi_label,combi_unique_num
ID,,,,,,,,,,,,,,,,,,,,,
0,0.753604,0.921160,DSSHDS,MGGY,0,BWY MGGY Call Combo 1 290719 PM,BWYa,CRAWLEY,F,C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-...,...,44100,DSSHDS,"[5.7770762, 7.997407]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",DS-SH-DS,DS-SH-DS SH-LH,0,0,DSSHDS SHSHLH,0
1,0.932017,1.367130,SHSHLH,MGGY,0,BWY MGGY Call Combo 1 290719 PM,BWYa,CRAWLEY,F,C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-...,...,44100,SHSHLH,"[1.8122675, 5.094298]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",SH-LH,DS-SH-DS SH-LH,1,1,DSSHDS SHSHLH,0
2,1.218085,1.308841,DS,MGGY,0,BWY MGGY Call Combo 1 300719 AM,BWYa,CRAWLEY,F,C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-...,...,44100,DS,"[9.353501, 10.055656]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",DS,DS SH-LH,2,0,DS USC,1


In [8]:
len(call_df)

561

## View UMAP

In [9]:
def norm(x):
    return (x-np.min(x)) / (np.max(x) - np.min(x))

In [10]:
#Get Specs
specs = list(call_df.spectrogram.values)
specs = [norm(i) for i in tqdm(specs)]

  0%|          | 0/561 [00:00<?, ?it/s]

In [11]:
from avgn.visualization.projections import scatter_spec

In [ ]:
nex = -1
scatter_spec(
    np.array(list(call_df['umap'].values)),
    specs,
    column_size=10,
    x_range = [-0.25,10.75],
    y_range = [1.5,12.5],
    pal_color="hls",
    color_points=False,
    enlarge_points=0,
    figsize=(12, 12),
    range_pad = 0.15,
    scatter_kwargs = {
        'labels': call_df.call_lab_simp.values,
        'alpha':0.8,
        's':10,
        'show_legend': True,
        "color_palette": 'magma',
    },
    matshow_kwargs = {
        'cmap': plt.cm.Greys
    },
    line_kwargs = {
        'lw':0.5,
        'ls':"dashed",
        'alpha':0.25,
    },
    draw_lines=True,
    n_subset= 1000,
    border_line_width = 3,
    

);

Appears to be some correlation with groups of similar call labels and distribution across clusters, as such we will create a new variable to describe these groups:
- calls containing an LH segment
- calls consisting of only an NL segment
- all other calls

In [13]:
call_df["simp"] = call_df["call_lab_simp"]

In [15]:
call_df.simp.unique()

array(['DS-SH-DS', 'SH-LH', 'DS', 'LH', 'NL-SH-DS', 'NL-DS',
       'SH-DS-SH-DS', 'SH-DS', 'NL-DS-SH-DS', 'LH-DS', 'SH-DS-SH-LH',
       'SH-DS-LH', 'SH-LH-DS', 'NL', 'SH-NL-DS', 'DS-SH-DS-SH-LH',
       'SH-DS-SH', 'SH'], dtype=object)

In [16]:
## Create conditions for all labels
cond1 = call_df['simp'] == 'DS-SH-DS'
cond2 = call_df['simp'] == 'SH-LH'
cond3 = call_df['simp'] == 'DS'
cond4 = call_df['simp'] == 'LH'
cond5 = call_df['simp'] == 'NL-SH-DS'
cond6 = call_df['simp'] == 'NL-DS'
cond7 = call_df['simp'] == 'SH-DS-SH-DS'
cond8 = call_df['simp'] == 'SH-DS'
cond9 = call_df['simp'] == 'NL-DS-SH-DS'
cond10 = call_df['simp'] == 'LH-DS'
cond11 = call_df['simp'] == 'SH-DS-SH-LH'
cond12 = call_df['simp'] == 'SH-DS-LH'
cond13 = call_df['simp'] == 'SH-LH-DS'
cond14 = call_df['simp'] == 'NL'
cond15 = call_df['simp'] == 'SH-DS-SH'
cond16 = call_df['simp'] == 'SH'
cond17 = call_df['simp'] == 'SH-NL-DS'
cond18 = call_df['simp'] == 'DS-SH-DS-SH-LH'

In [17]:
### Modify
call_df.loc[cond1, 'simp'] = 'Other Calls'
call_df.loc[cond2, 'simp'] = 'Contains LH Segment'
call_df.loc[cond3, 'simp'] = 'Other Calls'
call_df.loc[cond4, 'simp'] = 'Contains LH Segment'
call_df.loc[cond5, 'simp'] = 'Other Calls'
call_df.loc[cond6, 'simp'] = 'Other Calls'
call_df.loc[cond7, 'simp'] = 'Other Calls'
call_df.loc[cond8, 'simp'] = 'Other Calls'
call_df.loc[cond9, 'simp'] = 'Other Calls'
call_df.loc[cond10, 'simp'] = 'Contains LH Segment'
call_df.loc[cond11, 'simp'] = 'Contains LH Segment'
call_df.loc[cond12, 'simp'] = 'Contains LH Segment'
call_df.loc[cond13, 'simp'] = 'Contains LH Segment'
call_df.loc[cond14, 'simp'] = 'NL Segment Alone'
call_df.loc[cond15, 'simp'] = 'Other Calls'
call_df.loc[cond16, 'simp'] = 'Other Calls'
call_df.loc[cond17, 'simp'] = 'Other Calls'
call_df.loc[cond18, 'simp'] = 'Contains LH Segment'

In [18]:
call_df.simp.unique()

array(['Other Calls', 'Contains LH Segment', 'NL Segment Alone'],
      dtype=object)

In [19]:
len(call_df["simp"])

561

In [ ]:
nex = -1
scatter_spec(
    np.array(list(call_df['umap'].values)),
    specs,
    column_size=10,
    x_range = [0,10.25],
    y_range = [1.5,11.75],
    pal_color="hls",
    color_points=False,
    enlarge_points=0,
    figsize=(10, 10),
    range_pad = 0.15,
    scatter_kwargs = {
        'labels': call_df.simp.values,
        'alpha':0.8,
        's':10,
        'show_legend': True,
        "color_palette": 'magma',
    },
    matshow_kwargs = {
        'cmap': plt.cm.Greys
    },
    line_kwargs = {
        'lw':0.5,
        'ls':"dashed",
        'alpha':0.25,
    },
    draw_lines=True,
    n_subset= 1000,
    border_line_width = 3,
    

);


For the most part, this way of grouping calls appears to describe distinction between at least two of the major clusters. We will calculate silhouette score for labelling in this way (below) and can run further UMAPs on spectrograms within the groups to determine if it is an appropriate way to label the call groups, and also whether there is any further separation within each group (following notebooks). 

In [ ]:
nex = -1
scatter_spec(
    np.array(list(call_df['umap'].values)),
    specs,
    column_size=10,
    x_range = [-0.25,10.75],
    y_range = [1.5,12.5],
    pal_color="hls",
    color_points=False,
    enlarge_points=0,
    figsize=(15, 15),
    range_pad = 0.15,
    scatter_kwargs = {
        'labels': call_df.indv.values,
        'alpha':1,
        's': 8,
        'show_legend': True,
        "color_palette": 'magma',
    },
    matshow_kwargs = {
        'cmap': plt.cm.Greys
    },
    line_kwargs = {
        'lw':0.5,
        'ls':"dashed",
        'alpha':0.25,
    },
    draw_lines=True,
    n_subset= 1000,
    border_line_width = 3,
    

);

In [ ]:
nex = -1
scatter_spec(
    np.array(list(call_df['umap'].values)),
    specs,
    column_size=10,
    x_range = [-0.75,10.25],
    y_range = [1.5,12.5],
    pal_color="hls",
    color_points=False,
    enlarge_points=0,
    figsize=(10, 10),
    range_pad = 0.15,
    scatter_kwargs = {
        'labels': call_df.group.values,
        'alpha':1,
        's': 5,
        'show_legend': True,
        "color_palette": 'viridis',
    },
    matshow_kwargs = {
        'cmap': plt.cm.Greys
    },
    line_kwargs = {
        'lw':0.5,
        'ls':"dashed",
        'alpha':0.25,
    },
    draw_lines=True,
    n_subset= 1000,
    border_line_width = 3,
    

);

In [ ]:
nex = -1
scatter_spec(
    np.array(list(call_df['umap'].values)),
    specs,
    column_size=10,
    x_range = [0,10.25],
    y_range = [1.25,11.5],
    pal_color="hls",
    color_points=False,
    enlarge_points=0,
    figsize=(10, 10),
    range_pad = 0.15,
    scatter_kwargs = {
        'labels': call_df.location.values,
        'alpha':0.8,
        's': 10,
        'show_legend': True,
        "color_palette": 'tab20',
    },
    matshow_kwargs = {
        'cmap': plt.cm.Greys
    },
    line_kwargs = {
        'lw':0.5,
        'ls':"dashed",
        'alpha':0.25,
    },
    draw_lines=True,
    n_subset= 1000,
    border_line_width = 3,
    

);

## Silhouette Score

In [51]:
sexscore = silhouette_score(list(np.array(list(call_df['umap'].values))), labels = call_df.sex.values)
indvscore = silhouette_score(list(np.array(list(call_df['umap'].values))), labels = call_df.indv.values)
locscore = silhouette_score(list(np.array(list(call_df['umap'].values))), labels = call_df.location.values)
groupscore = silhouette_score(list(np.array(list(call_df['umap'].values))), labels = call_df.group.values)
callscore = silhouette_score(list(np.array(list(call_df['umap'].values))), labels = call_df.call_lab_simp.values)
simpscore = silhouette_score(list(np.array(list(call_df['umap'].values))), labels = call_df.simp.values)

In [ ]:
calldata = {'Test': ['S'], 'Call_Lab': [callscore], 'Simp':[simpscore], 'Indv':[indvscore], 'Group':[groupscore], 
            'Study Site':[locscore], 'Sex':[sexscore]}
df = pd.DataFrame(calldata)
df

## Kruskal-Wallis H Test

In [58]:
samples = silhouette_samples(list(np.array(list(call_df['umap'].values))), labels = call_df.call_lab_simp.values)
chance_samples = silhouette_samples(list(np.array(list(call_df['umap'].values))), labels = np.random.permutation(call_df.call_lab_simp.values))

In [ ]:
KWcall = kruskal(samples, chance_samples)
KWcall

In [63]:
samples = silhouette_samples(list(np.array(list(call_df['umap'].values))), labels = call_df.simp.values)
chance_samples = silhouette_samples(list(np.array(list(call_df['umap'].values))), labels = np.random.permutation(call_df.simp.values))

In [ ]:
KWsimp = kruskal(samples, chance_samples)
KWsimp

In [81]:
samples = silhouette_samples(list(np.array(list(call_df['umap'].values))), labels = call_df.indv.values)
chance_samples = silhouette_samples(list(np.array(list(call_df['umap'].values))), labels = np.random.permutation(call_df.indv.values))

In [ ]:
KWindv = kruskal(samples, chance_samples)
KWindv

In [83]:
samples = silhouette_samples(list(np.array(list(call_df['umap'].values))), labels = call_df.group.values)
chance_samples = silhouette_samples(list(np.array(list(call_df['umap'].values))), labels = np.random.permutation(call_df.group.values))

In [ ]:
KWgroup = kruskal(samples, chance_samples)
KWgroup

In [93]:
samples = silhouette_samples(list(np.array(list(call_df['umap'].values))), labels = call_df.location.values)
chance_samples = silhouette_samples(list(np.array(list(call_df['umap'].values))), labels = np.random.permutation(call_df.location.values))

In [ ]:
KWloc = kruskal(samples, chance_samples)
KWloc

In [91]:
samples = silhouette_samples(list(np.array(list(call_df['umap'].values))), labels = call_df.sex.values)
chance_samples = silhouette_samples(list(np.array(list(call_df['umap'].values))), labels = np.random.permutation(call_df.sex.values))

In [ ]:
KWsex = kruskal(samples, chance_samples)
KWsex

In [ ]:
calldata = {'Test': ['S', 'KWstat', 'KWpval'], 'Call':[callscore, KWcall.statistic, KWcall.pvalue], 
           'Simp':[simpscore, KWsimp.statistic, KWsimp.pvalue], 'Indv':[indvscore, KWindv.statistic, KWindv.pvalue],
           'Group':[groupscore, KWgroup.statistic, KWgroup.pvalue], 'Study Site':[locscore, KWloc.statistic, KWloc.pvalue],
            'Sex':[sexscore, KWsex.statistic, KWsex.pvalue']]}
df = pd.DataFrame(calldata)
df

In [100]:
#save df
save_loc = DATA_DIR / DATASET_ID / DT_ID /  'call_umap_grouped.pickle'
ensure_dir(save_loc.as_posix())
call_df.to_pickle(save_loc)